In [1]:
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
import numpy as np
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os
from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Proj GREEQ 1.05/crupbl"

In [ ]:
train = path+'/Train'
img_width, img_height, channels = 150, 150, 3 # you can try different sizes
input_shape = (img_width, img_height, 3)
batch_size = 15
nb_train_samples = 179
nb_validation_samples = nb_train_samples//5

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   vertical_flip=True,
                                   horizontal_flip=True,
                                   validation_split=0.2)  

val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)


train_data = train_datagen.flow_from_directory(train, 
                                               target_size=(img_width, img_height), 
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset = 'training') 

val_data = val_datagen.flow_from_directory(train, 
                                           target_size=(img_width, img_height), 
                                           batch_size=batch_size,
                                           class_mode='categorical',
                                           shuffle=False,
                                           subset = 'validation')

In [ ]:
train_data.class_indices

{'grechka': 0, 'muka': 1, 'ris': 2}

In [ ]:
model = Sequential()
model.add(Conv2D(4, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(15,activation='relu'))
model.add(Dense(3,activation='softmax'))

Скриптина принимает на вход папку на чем обучать abs.path, обучается, по а вторая скриптина берет картинку выплевывает что на картинке 


In [ ]:
model.compile( loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)


In [ ]:
history = model.fit(
    train_data,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=20, #try different number of epochs: 10, 15, 20; check the loss and accuracy;
    validation_data=val_data,
    callbacks = [es],
    )

Epoch 1/20
11/11 [==============================] - 98s 9s/step - loss: 1.2150 - accuracy: 0.4147 - val_loss: 0.9965 - val_accuracy: 0.4091
Epoch 2/20
11/11 [==============================] - 9s 756ms/step - loss: 0.9944 - accuracy: 0.3787 - val_loss: 0.9635 - val_accuracy: 0.4091
Epoch 3/20
11/11 [==============================] - 9s 771ms/step - loss: 0.8435 - accuracy: 0.5162 - val_loss: 0.8404 - val_accuracy: 0.4773
Epoch 4/20
11/11 [==============================] - 9s 787ms/step - loss: 0.7727 - accuracy: 0.6623 - val_loss: 0.6361 - val_accuracy: 0.7045
Epoch 5/20
11/11 [==============================] - 9s 775ms/step - loss: 0.6770 - accuracy: 0.7186 - val_loss: 1.0483 - val_accuracy: 0.5682
Epoch 6/20
11/11 [==============================] - 9s 770ms/step - loss: 0.4784 - accuracy: 0.7986 - val_loss: 0.7522 - val_accuracy: 0.6364
Epoch 7/20
11/11 [==============================] - 9s 781ms/step - loss: 0.6544 - accuracy: 0.7000 - val_loss: 0.6347 - val_accuracy: 0.6818
Epoch 8/

In [ ]:
model.save(path+'/my_model.h5')

In [ ]:
model.save(path+'/my_model.tf')

In [ ]:
# summarize history for loss
import matplotlib.pyplot as plt 
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
for folder, subfolders, files in os.walk(path+"/Test"):
  for file in files:
    img = image.load_img(os.path.join(folder,file), target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = int(model.predict(x)[0][0]) 
    print(os.path.split(folder)[-1],class_names[feature])
